In [ ]:
import math
import numpy as np
import pandas as pd

This is our dataset:

In [ ]:
df = pd.read_csv('ObesityDataSet_raw_and_data_sinthetic.csv')

In [ ]:
y: pd.Series = df['NObeyesdad'].str.startswith('Obesity').astype(int)
del df['NObeyesdad']
del df['Height']
del df['Weight']
X: pd.DataFrame = df

In [ ]:
def evaluate(y_actual, y_pred):
    print('accuracy:', accuracy_score(y_actual, y_pred))
    print('precision:', precision_score(y_actual, y_pred))
    print('recall:', recall_score(y_actual, y_pred))
    print('f1:', f1_score(y_actual, y_pred))
    print('confusion matrix:')
    print(confusion_matrix(y_actual, y_pred))

In [ ]:
def preprocessing_perceptron(X: pd.DataFrame) -> pd.DataFrame:
    freq_map = {'no': 0, 'Sometimes': 1,
                'Frequently': 2, 'Always': 3}

    yes_no_map = {'yes': 1, 'no': 0}

    X['Gender'] = X['Gender'].map({'Male': 1, 'Female': 0})
    X['family_history_with_overweight'] = X['family_history_with_overweight'].map(
        yes_no_map)
    X['FAVC'] = X['FAVC'].map(yes_no_map)
    X['CAEC'] = X['CAEC'].map(freq_map)
    X['SMOKE'] = X['SMOKE'].map(yes_no_map)
    X['SCC'] = X['SCC'].map(yes_no_map)
    X['CALC'] = X['CALC'].map(freq_map)
    X['MTRANS'] = X['MTRANS'].map(
        {'Automobile': 0, 'Motorbike': 0, 'Public_Transportation': 0, 'Walking': 1, 'Bike': 1})

    normalized_X = (X-X.min())/(X.max()-X.min())
    return normalized_X

This is the implementation of perceptron:

In [ ]:
def net(X: np.ndarray, W: np.ndarray) -> float:
    return np.dot(X, W)


def unit_step(x) -> float:
    if x > 0:
        return 1
    else:
        return 0


def sigmoid(x) -> float:
    return 1 / (1 + math.exp(-x))


class MyPerceptron:
    def fit(self, X: np.ndarray, y: np.ndarray) -> None:
        assert X.ndim == 2
        assert y.ndim == 1
        assert X.shape[0] == y.shape[0]

        self.W: np.ndarray = np.zeros(X.shape[1] + 1)  # weights

        X0 = np.ones((X.shape[0], 1))
        X = np.hstack((X0, X))

        for i in range(X.shape[0]):
            y_pred: float = sigmoid(net(X[i, :], self.W))
            error: float = y[i] - y_pred
            self.W += error * X[i, :]

    def predict(self, X: np.ndarray) -> np.ndarray:
        X0 = np.ones((X.shape[0], 1))
        X = np.hstack((X0, X))
        v_unit_step = np.vectorize(unit_step)
        return v_unit_step(net(X, self.W))

In [ ]:
from sklearn.linear_model import Perceptron
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix


def learn_perceptron(X: pd.DataFrame, y: pd.Series):
    X = X.to_numpy()
    y = y.to_numpy()

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.1, random_state=0)

    clf = Perceptron(tol=1e-3, random_state=0)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    evaluate(y_test, y_pred)

    clf = MyPerceptron()
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    evaluate(y_test, y_pred)

In [ ]:
X_perceptron = preprocessing_perceptron(X)
learn_perceptron(X_perceptron, y)